# plot each lidar snow, summed sd, r2

In [40]:
import xarray as xr
import numpy as np
import pandas as pd
import rioxarray as rxa
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib as mpl
import pandas as pd

from scipy.ndimage import gaussian_filter

from stats import clean_xs_ys

# loop through each flight pair and calculate rmse, r2, plot
lidar = None

# check if we are on borah or local
import socket
hn = socket.gethostname()
lidar_dir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar')
if hn == 'Zachs-MacBook-Pro.local':
    lidar_dir = Path('/Users/zachkeskinen/Desktop/')
for img_type in ['int','unw']:
    print(img_type)
    # for lidar filepath
    for lidar_fp in lidar_dir.glob('*.sd.nc'):
        print(lidar_fp)
        lidar = xr.open_dataset(lidar_fp)

        # single times are strings not lists
        if isinstance(lidar.attrs['lidar_times'] , str):
            lidar.attrs['lidar_times'] = [lidar.attrs['lidar_times']]

        for t in lidar.attrs['lidar_times']:

            t = pd.to_datetime(t)
            # slice images that occured that year
            ds = lidar.sel(time = slice(t - pd.Timedelta('180 days'), t + pd.Timedelta('90 days')))

            increasing = ds.where(ds.snotel_dSWE > 0)
            # increasing = increasing.where(increasing['232-cor'] > 0.4)
            
            cum_sd232 = increasing[f'232-sd_delta_{img_type}'].sum(dim = 'time', skipna = True).where(~ds['lidar-sd'].sel(time = t).isnull())
            cum_sd232 = cum_sd232.where(cum_sd232 != 0)
            cum_sd052 = increasing[f'052-sd_delta_{img_type}'].sum(dim = 'time', skipna = True).where(~ds['lidar-sd'].sel(time = t).isnull())

            vmax = cum_sd232.sel(band = 'VV').quantile(0.9)

            if cum_sd052.sum() > 0:
                fig, axes = plt.subplots(1, 4, figsize = (20,10))
                data232 = cum_sd232.sel(band = 'VV')
                data232.data = gaussian_filter(data232, 2)
                data232.plot(vmin = 0, vmax = vmax, ax = axes[1], cmap = 'viridis')
                data = cum_sd052.sel(band = 'VV')
                data.data = gaussian_filter(data, 2)
                data.plot(vmin = 0, vmax = vmax, ax = axes[2], cmap = 'viridis')
                ax = axes[3]
            else:
                fig, axes = plt.subplots(1, 3, figsize = (20,10))
                data232 = cum_sd232.sel(band = 'VV')
                data232.data = gaussian_filter(data232, 2)
                data232.plot(vmin = 0, vmax = vmax, ax = axes[1], cmap = 'viridis')
                ax = axes[2]

            ds['lidar-sd'].sel(time = t).plot(vmin = 0, vmax = 3, ax = axes[0])

            xs = lidar['lidar-sd'].sel(time = t).values.ravel()
            ys = data232.values.ravel()

            xs, ys = clean_xs_ys(xs, ys, clean_zeros = True)

            range = [[-.2, 5.5], [-.2, np.max(ys) + 0.3]]
            ax.hist2d(xs, ys, bins = 100, norm=mpl.colors.LogNorm(), cmap=mpl.cm.inferno, range = range)

            from scipy.stats import pearsonr
            r, p = pearsonr(xs, ys)
            ax.text(.01, .99, f'r: {r:.2}\nn = {len(ys):.2e}', ha = 'left', va = 'top', transform = ax.transAxes)
            plt.tight_layout()
            # plt.show()
            plt.savefig(f"/bsuhome/zacharykeskinen/uavsar-validation/figures/lidar/new/{img_type}_{lidar.attrs['site']}_{t.strftime('%Y-%m-%d')}.png")
            plt.close()

/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar/Mores.sd.nc
/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar/Banner.sd.nc
/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar/Dry_Creek.sd.nc


In [1]:
def get_r(ds, lidar):
    xs = lidar.values.ravel()
    ys = ds.values.ravel()

    xs, ys = clean_xs_ys(xs, ys, clean_zeros = True)
    r, p = pearsonr(xs, ys)
    return r

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import rioxarray as rxa
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib as mpl
import pandas as pd

from scipy.ndimage import gaussian_filter

from stats import clean_xs_ys
from scipy.stats import pearsonr
from uavsar_pytools.snow_depth_inversion import depth_from_phase

# import warnings
# warnings.filterwarnings("ignore")

# loop through each flight pair and calculate rmse, r2, plot
lidar = None

# check if we are on borah or local
import socket
hn = socket.gethostname()
lidar_dir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar')
if hn == 'Zachs-MacBook-Pro.local':
    lidar_dir = Path('/Users/zachkeskinen/Desktop/')

for img_type in ['int','unw']:
    print(img_type)
    # for lidar filepath
    for lidar_fp in lidar_dir.glob('*.sd.nc'):
        print(lidar_fp)
        lidar = xr.open_dataset(lidar_fp)

        # single times are strings not lists
        if isinstance(lidar.attrs['lidar_times'] , str):
            lidar.attrs['lidar_times'] = [lidar.attrs['lidar_times']]

        for t in lidar.attrs['lidar_times']:

            t = pd.to_datetime(t)
            lidar_ds = lidar['lidar-sd'].sel(time = t)
            # slice images that occured that year
            ds = lidar.sel(time = slice(t - pd.Timedelta('180 days'), t + pd.Timedelta('90 days')))
            ds = ds.where(ds.snotel_dSWE > 0)
            ds = ds.sel(band = 'VV')
            ds['sd-delta'] = xr.zeros_like(ds['232-int'])
            
            for ts in ds.time:
                ds_ts = ds.sel(time = ts)
                snotel_den = np.abs((ds_ts['snotel_dSWE'] / ds_ts['snotel_dSD'] * 997).data.ravel()[0])
                if ds_ts[f'232-{img_type}'].sum() > 0:
                    dsd = depth_from_phase(ds_ts[f'232-{img_type}'], inc_angle = ds_ts['232-inc'], density = snotel_den)
                    dsd = dsd + ((ds_ts['snotel_dSWE'] / (snotel_den / 997)) - dsd.mean())
                    r = get_r(dsd, lidar_ds)
                    if r > 0.1:
                        ds['sd-delta'].loc[dict(time = ts)] = dsd

            cum_sd232 = ds['sd-delta'].sum(dim = 'time', skipna = True).where(~lidar_ds.isnull())
            cum_sd232 = cum_sd232.where((cum_sd232 < 0.01) | (cum_sd232 > 0.01)) 
            lidar_ds = lidar_ds.where(lidar_ds > 0.001)

            vmax = cum_sd232.quantile(0.9)
            vmin = cum_sd232.quantile(0.1)
            # cum_sd232.data = gaussian_filter(cum_sd232, 2)
            fig, axes = plt.subplots(1, 3, figsize = (20,10))

            lidar_ds.plot(vmin = 0, vmax = 3, ax = axes[0])
            cum_sd232.plot(vmin = vmin, vmax = vmax, ax = axes[1], cmap = 'viridis')

            xs = lidar_ds.values.ravel()
            ys = cum_sd232.values.ravel()

            xs, ys = clean_xs_ys(xs, ys, clean_zeros = True)

            range = [[-.2, 5.5], [-.2, np.max(ys) + 0.3]]
            axes[2].hist2d(xs, ys, bins = 100, norm=mpl.colors.LogNorm(), cmap=mpl.cm.inferno, range = range)

            r, p = pearsonr(xs, ys)
            axes[2].text(.01, .99, f'r: {r:.2}\nn = {len(ys):.2e}', ha = 'left', va = 'top', transform = axes[2].transAxes)
            plt.tight_layout()
            # plt.show()
            plt.savefig(f"/bsuhome/zacharykeskinen/uavsar-validation/figures/lidar/snotel/{img_type}_{lidar.attrs['site']}_{t.strftime('%Y-%m-%d')}.png")
            plt.close()

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
from stats import get_stats, clean_xs_ys

df = pd.read_csv('../../results/lidar/slope.csv', index_col=['stat', 'loc','date'])

fig, ax = plt.subplots(figsize = (12,8))
df.loc[('r')].T.rolling(3).mean().plot(ax = ax)
df.loc[('r')].T.rolling(3).mean().mean(axis = 1).plot(ax = ax, color = 'black', linewidth = 5)

In [10]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
from tqdm import tqdm

from pathlib import Path
from stats import get_stats, clean_xs_ys

from scipy.ndimage import gaussian_filter

from xrspatial import slope, aspect, curvature

snotel_locs = {'Dry_Creek.sd': [-116.09685, 43.76377], 'Banner.sd': [-115.23447, 44.30342], 'Mores.sd': [-115.66588, 43.932]}

import socket
hn = socket.gethostname()
lidar_dir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar')
if hn == 'Zachs-MacBook-Pro.local':
    lidar_dir = Path('/Users/zachkeskinen/Desktop/')

out_dir = Path('/bsuhome/zacharykeskinen/uavsar-validation/results/lidar')

## Slope

res = pd.DataFrame(index = pd.MultiIndex.from_arrays([[],[],[]], names=('stat', 'loc', 'date')))

for lidar_fp in lidar_dir.glob('*.sd.nc'):
    print(lidar_fp)

    lidar = xr.open_dataset(lidar_fp)
    
    if isinstance(lidar.attrs['lidar_times'] , str):
        lidar.attrs['lidar_times'] = [lidar.attrs['lidar_times']]
    sntl_x, sntl_y = snotel_locs[lidar_fp.stem]
    
    for t in lidar.attrs['lidar_times']:
        t = pd.to_datetime(t)

        ds_orig = lidar.sel(time = slice(t - pd.Timedelta('180 days'), t))
        ds_orig = ds_orig.sel(band = 'VV')

        lidar_slope = slope(lidar['lidar-dem'].rio.reproject('EPSG:32611')).rio.reproject_match(lidar['lidar-sd'].sel(time = t))

        delta = 1 # degrees
        for high in tqdm(np.arange(delta, lidar_slope.max(), delta)):
            res = res.sort_index()
            low = high - delta
            print(f'Running {low} to {high}')
            
            ds = ds_orig.copy(deep = True)

            lidar_sd = ds['lidar-sd'].sel(time = t).where((lidar_slope > low) & (lidar_slope < high))
            # lidar_sd.data = gaussian_filter(lidar_sd, 3)

            ds = ds.sel(time = ds.snotel_dSWE > 0)
            
            sum = ds['232-sd_delta_int'].sum(dim = 'time')
            sum.data = gaussian_filter(sum, 3)
            print(lidar_sd.mean())
            print(sum.mean())

            xs, ys = clean_xs_ys(lidar_sd.values.ravel(), sum.values.ravel(), clean_zeros = True)

            if len(xs) < 2 or len(ys) < 2:
                print(f"No values for {low}-{high}.")
                rmse, r, n =  np.nan, np.nan, np.nan
            else:
                print(f"Values found")
                rmse, r, n = get_stats(xs, ys)
            
            loc = lidar_fp.name.replace('.sd.nc', '')
            date = t.strftime('%Y-%m-%d')

            for stat_name, stat in zip(['n', 'r', 'rmse'], [n, r, rmse]):
                res.loc[(stat_name, loc, date), f'{low}-{high}'] = stat

# res.to_csv(out_dir.joinpath('slope.csv'))

/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/lidar/Mores.sd.nc


  0%|          | 0/75 [00:00<?, ?it/s]

Running 0.0 to 1.0
<xarray.DataArray 'lidar-sd' ()>
array(1.8611681, dtype=float32)
Coordinates:
    time         datetime64[ns] 2020-02-09
    band         <U2 'VV'
    spatial_ref  int64 0
    time2        datetime64[ns] ...
    snotel_dSWE  float64 ...
    snotel_dSD   float64 ...
<xarray.DataArray '232-sd_delta_int' ()>
array(0.55530936, dtype=float32)
Coordinates:
    band         <U2 'VV'
    spatial_ref  int64 ...
Values found


  1%|▏         | 1/75 [00:07<09:42,  7.87s/it]

Running 1.0 to 2.0
<xarray.DataArray 'lidar-sd' ()>
array(1.7800655, dtype=float32)
Coordinates:
    time         datetime64[ns] 2020-02-09
    band         <U2 'VV'
    spatial_ref  int64 0
    time2        datetime64[ns] ...
    snotel_dSWE  float64 ...
    snotel_dSD   float64 ...
<xarray.DataArray '232-sd_delta_int' ()>
array(0.55530936, dtype=float32)
Coordinates:
    band         <U2 'VV'
    spatial_ref  int64 ...


  1%|▏         | 1/75 [00:10<13:04, 10.60s/it]


KeyboardInterrupt: 